In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: c:\Users\User\.pyenv\pyenv-win\versions\3.12.1\python.exe -m pip install --upgrade pip


In [2]:
pip install fsspec


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: c:\Users\User\.pyenv\pyenv-win\versions\3.12.1\python.exe -m pip install --upgrade pip


In [3]:
pip install s3fs

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: c:\Users\User\.pyenv\pyenv-win\versions\3.12.1\python.exe -m pip install --upgrade pip


In [8]:
pip install pyarrow


  Obtaining dependency information for pyarrow from https://files.pythonhosted.org/packages/74/39/323621402c2b1ce7ba600d03c81cf9645b862350d7c495f3fcef37850d1d/pyarrow-18.0.0-cp312-cp312-win_amd64.whl.metadata
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.1 MB 320.0 kB/s eta 0:01:19
   ---------------------------------------- 0.0/25.1 MB 326.8 kB/s eta 0:01:17
   ---------------------------------------- 0.1/25.1 MB 798.9 kB/s eta 0:00:32
    --------------------------------------- 0.3/25.1 MB 1.5 MB/s eta 0:00:17
   - -------------------------------------- 0.7/25.1 MB 2.6 MB/s eta 0:00:10
   - -------------------------------------- 1.1/25.1 MB 3.8 MB/s eta 0:00:07
   -- ------------------------------------- 1.7/25.1 MB 5.3 MB/s eta 0:00:05
   -- ------------------------------------- 1.7/25.1 MB 5.3 MB/s eta 0:00:05
   --- ---------------------


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: c:\Users\User\.pyenv\pyenv-win\versions\3.12.1\python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd

# Configuração para acesso ao MinIO
s3_url = "http://54.161.237.227:9000/"
access_key = "minio_access_key"
secret_key = "minio_secret_key"

storage_options = {
    "key": access_key,
    "secret": secret_key,
    "client_kwargs": {
        "endpoint_url": s3_url
    }
}

file_path = 's3://bronze/data/order-details.csv'

# Tentar ler o arquivo CSV com tratamento para aspas e delimitadores, e lidando com linhas ruins
try:
    df = pd.read_csv(
        file_path,
        sep=',',
        quotechar='"',  # Assegura que as vírgulas dentro de aspas sejam tratadas corretamente
        storage_options=storage_options,
        on_bad_lines='skip'  # Pula linhas que não têm o número correto de delimitadores
    )
    display(df)
except Exception as e:
    print("Erro ao ler o arquivo:", e)


,orderID,productID,unitPrice,quantity,discount
0,10248,11,14.00,12,0.00
1,10248,42,9.80,10,0.00
2,10248,72,34.80,5,0.00
3,10249,14,18.60,9,0.00
4,10249,51,42.40,40,0.00
...,...,...,...,...,...
2150,11077,64,33.25,2,0.03
2151,11077,66,17.00,1,0.00
2152,11077,73,15.00,2,0.01
2153,11077,75,7.75,4,0.00


In [5]:
df_renamed = df.rename(columns={
    'productID': 'cd_produto'
                   })

In [6]:

df_formatted = df_renamed.astype({
    'orderID': 'int',
    'cd_produto': 'int',
    'unitPrice': 'float',
    'quantity': 'int',
    'discount': 'float'
})

display(df_formatted)

,orderID,cd_produto,unitPrice,quantity,discount
0,10248,11,14.00,12,0.00
1,10248,42,9.80,10,0.00
2,10248,72,34.80,5,0.00
3,10249,14,18.60,9,0.00
4,10249,51,42.40,40,0.00
...,...,...,...,...,...
2150,11077,64,33.25,2,0.03
2151,11077,66,17.00,1,0.00
2152,11077,73,15.00,2,0.01
2153,11077,75,7.75,4,0.00


In [9]:
parquet_path = 's3://silver/orders.parquet'


try:
    df.to_parquet(
        parquet_path,
        engine='pyarrow',
        index=False,       
        storage_options=storage_options
    )
    print("Arquivo Parquet salvo com sucesso em:", parquet_path)
except Exception as e:
    print("Erro ao salvar o arquivo Parquet:", e)


Arquivo Parquet salvo com sucesso em: s3://silver/orders.parquet
